In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
from utils import *

2024-02-11 18:25:35.305571: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-11 18:25:35.308780: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-11 18:25:35.351070: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-11 18:25:35.351096: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-11 18:25:35.352541: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
import random
import json

In [3]:
# from transformers import AutoModelForCausalLM, AutoTokenizer

# device = "cuda" # the device to load the model onto

# model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
# tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")


# model.to(device)

In [4]:
# messages = [
#     {"role": "user", "content": "You are a helpful abiding assistant. Your inputs will consist of a question and a correct answer, and a answer from a model. Your response should start with a single word either a yes if the model's answer means the correct answer, else the starting word should be no."},
#     {"role": "assistant", "content": "Okay, I will do that."},
#     {"role": "user", "content": "'Question: For which of these two scenarios does the main character (who uses I/me/my) do something clearly morally wrong, according to ordinary moral standards in the US as of 2020? Scenario 1 | I bared my entire naked body at the beach. Scenario 2 | I told her it was safe to jump off the couch. Ground Truth: Wrong, Not wrong. Model Answer: Not wrong, Wrong"}
# ]

# encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

# model_inputs = encodeds.to(device)

# generated_ids = model.generate(model_inputs, max_new_tokens=1, do_sample=False)
# decoded = tokenizer.batch_decode(generated_ids)
# #print(decoded[0])
# tokenizer.batch_decode(generated_ids[:, model_inputs.size(1):])[0]

In [5]:
# generated_ids.shape, model_inputs.shape, len(decoded[0]), len(tokenizer.batch_decode(model_inputs)[0]), generated_ids[:, model_inputs.size(1):].shape, len(tokenizer.batch_decode(generated_ids[:, model_inputs.size(1):])[0]), tokenizer.batch_decode(generated_ids[:, model_inputs.size(1):])[0]

In [6]:
# decoded[0][-3:]

In [7]:
# model.generation_config

In [8]:
all_data = []
with open('perturbed1.txt', 'r') as f:
    all_data = f.readlines()
print(len(all_data))
with open('wrong.txt', 'r') as f:
    all_data += f.readlines()
print(len(all_data))
random.shuffle(all_data)

501
1001


In [10]:
json.loads(all_data[3])

{'question': 'The flight of a blimp best illustrates',
 'correct_answer': 'Archimedes’ principle',
 'perturbed_answer': "buoyancy, which is explained by Archimedes' principle.",
 'options': 'Pascal’s principle;Bernoulli’s principle;Boyle’s law',
 'perturbed': 'yes'}

In [11]:
checker = MistralChecker(model_path='mistralai/Mistral-7B-Instruct-v0.2', device='cuda')

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [12]:
data_test = json.loads(all_data[3])
data_test

{'question': 'The flight of a blimp best illustrates',
 'correct_answer': 'Archimedes’ principle',
 'perturbed_answer': "buoyancy, which is explained by Archimedes' principle.",
 'options': 'Pascal’s principle;Bernoulli’s principle;Boyle’s law',
 'perturbed': 'yes'}

In [13]:
checker.raw_checker(question=data_test['question'], correct_ans=data_test['correct_answer'], model_ans=data_test['perturbed_answer'])

(1, None)

In [14]:
correct = 0
total = 0
diff = 0
errors = []
for i, data in enumerate(all_data):
    try:
        if i % 100 == 1:
            print(correct/(total+1e-3), correct, total)
        data = json.loads(data)
        messages = [
            {"role": "user", "content": "context: You are a helpful abiding assistant. Your inputs will consist of a question and a correct answer, and a answer from a model. Your response should start with a single word either a yes if the model's answer means the correct answer, else the starting word should be no."},
            {"role": "assistant", "content": "Okay, I will do that."},
        ]
        expected_checker_ans = 1
        if 'perturbed_answer' in data:
            ans = checker.raw_checker(question=data['question'], correct_ans=data['correct_answer'], model_ans=data['perturbed_answer'])
            ans = int(ans[0])
        else:
            ans = checker.raw_checker(question=data['question'], correct_ans=data['correct_answer'], model_ans=data['wrong_answer'])
            expected_checker_ans = 0
            ans = int(ans[0])
        
        # print(decoded, expected_checker_ans)
        if ans == expected_checker_ans:
            correct += 1
        total += 1
    except Exception as e:
        print(correct/total, correct, total)
        error = e
        errors.append(data)
        continue
correct/total, correct, total, errors

0.0 0 1
0.8514767180522965 86 101
0.8805926338674932 177 201
0.8737512499958473 263 301
0.8678282597799009 348 401
0.8602777239965589 431 501
0.8618954045001589 518 601
0.8559188931256875 600 701
0.8501862045116049 681 801
0.8523852914702648 768 901


(0.8521478521478522, 853, 1001, [])